# Fine-tuning Qwen2.5-0.5B pour post-traitement dictée FR

**Objectif** : Entraîner Qwen2.5-0.5B-Instruct avec LoRA pour corriger les sorties brutes de Whisper en français.

**Tâches couvertes** :
- Task 20 — Génération du dataset (paires brut Whisper → texte corrigé)
- Task 21 — Fine-tuning LoRA/QLoRA sur Qwen2.5-0.5B
- Task 22 — Export du modèle au format GGUF pour intégration dans l'app

## Instructions
1. **Runtime** → Modifier le type d'exécution → GPU (T4 gratuit ou mieux)
2. Cliquer **Exécution** → **Tout exécuter**
3. À la fin (~2-4h sur T4), télécharger `qwen25-dictation-fr-q4.gguf`
4. Placer le fichier dans `~/Library/Application Support/dictation-ia/models/`

---
**Prérequis** : Aucun. Tout est automatisé.

**Coût** : 0€ (Colab gratuit T4) ou ~3-5€ (Colab Pro A100, 4× plus rapide)

## 0. Vérification GPU

In [ ]:
import subprocess
result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], 
                       capture_output=True, text=True)
if result.returncode == 0:
    gpu_info = result.stdout.strip()
    print(f'✅ GPU détecté : {gpu_info}')
else:
    raise RuntimeError('❌ Aucun GPU détecté. Allez dans Runtime → Modifier le type d\'exécution → GPU')

## 1. Installation des dépendances

In [ ]:
%%capture
!pip install -q \
    transformers==4.46.0 \
    peft==0.13.0 \
    trl==0.12.0 \
    datasets==3.1.0 \
    bitsandbytes==0.44.1 \
    accelerate==1.1.0 \
    llama-cpp-python==0.3.2 \
    huggingface_hub

print('✅ Dépendances installées')

## 2. Configuration

In [ ]:
# ── Configuration ─────────────────────────────────────────────────────────────
MODEL_ID       = 'Qwen/Qwen2.5-0.5B-Instruct'
OUTPUT_DIR     = '/content/qwen25-dictation-fr'
GGUF_NAME      = 'qwen25-dictation-fr-q4.gguf'
MAX_SEQ_LEN    = 256   # Phrases dictées courtes — économise la VRAM
EPOCHS         = 3
BATCH_SIZE     = 8
GRAD_ACCUM     = 4     # Batch effectif = 32
LEARNING_RATE  = 2e-4
LORA_R         = 16
LORA_ALPHA     = 32
LORA_DROPOUT   = 0.05
# ──────────────────────────────────────────────────────────────────────────────

import os, json, random
from pathlib import Path

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
print(f'✅ Config OK — modèle cible : {MODEL_ID}')

## 3. Génération du dataset (Task 20)

Génère ~10 000 paires *(sortie brute Whisper → texte corrigé)* en français.
Les erreurs typiques de Whisper FR sont simulées : fillers, élisions mal formées, ponctuation manquante, bégaiements.

In [ ]:
import re, random

# ── Corpus de phrases françaises propres ──────────────────────────────────────
CLEAN_SENTENCES = [
    # Quotidien
    "Je voudrais réserver une table pour deux personnes ce soir.",
    "Peux-tu m'envoyer le document par email s'il te plaît ?",
    "La réunion est prévue pour demain matin à neuf heures.",
    "Il faudrait qu'on discute de ce projet avant la fin de la semaine.",
    "J'ai besoin d'aide pour configurer mon ordinateur.",
    "Le rapport doit être remis au plus tard vendredi.",
    "Tu peux me rappeler dans une heure ?",
    "On se retrouve à la gare à midi moins le quart.",
    "N'oublie pas d'acheter du pain sur le chemin du retour.",
    "La présentation s'est très bien passée, tout le monde était satisfait.",
    # Professionnel
    "Suite à notre échange téléphonique, je vous adresse ce compte-rendu.",
    "Merci de bien vouloir confirmer votre présence avant jeudi.",
    "Je reste disponible pour toute question complémentaire.",
    "Le budget prévisionnel a été validé par la direction.",
    "Nous devons impérativement respecter les délais contractuels.",
    "L'équipe technique travaille sur la résolution du problème.",
    "Ce point sera abordé lors de la prochaine réunion de suivi.",
    "Je vous transmets les documents demandés en pièce jointe.",
    "Pouvez-vous me confirmer les spécifications techniques requises ?",
    "La mise en production est planifiée pour le premier du mois prochain.",
    # Technique
    "Il faut mettre à jour la base de données avant le déploiement.",
    "Le bug est lié à une mauvaise gestion des erreurs dans le module audio.",
    "On utilise une architecture microservices avec des conteneurs Docker.",
    "Les tests unitaires doivent couvrir au moins quatre-vingts pourcent du code.",
    "J'ai trouvé la source du problème dans la fonction de traitement audio.",
    "La latence du pipeline doit rester inférieure à deux secondes.",
    "Le modèle de langage tourne en local sur le processeur neural Apple.",
    "Il faudrait optimiser l'algorithme de détection de fin de parole.",
    "Les logs indiquent une fuite mémoire dans le gestionnaire de ressources.",
    "On peut améliorer les performances en utilisant le cache de second niveau.",
    # Conversation
    "Franchement je pense que cette solution est la meilleure.",
    "Tu vois ce que je veux dire, c'est vraiment important pour nous.",
    "En fait la situation est plus compliquée que ce qu'on croyait.",
    "Je ne sais pas trop comment aborder ce sujet avec lui.",
    "C'est exactement ça, tu as tout à fait compris.",
    "Attends, laisse-moi réfléchir deux secondes avant de répondre.",
    "Honnêtement, je ne suis pas sûr que ce soit la meilleure approche.",
    "On pourrait essayer d'une autre façon, ça me semble plus simple.",
    "J'ai l'impression qu'on tourne en rond sur ce sujet.",
    "Ce que tu proposes est intéressant mais ça demande plus de temps.",
]

# ── Simulation des erreurs typiques de Whisper FR ─────────────────────────────
FILLERS = ['euh ', 'heu ', 'bah ', 'ben ', 'du coup ', 'genre ', 'voilà ', 
           'en fait ', 'quoi ', 'hein ', 'bon ben ', 'disons ']

def inject_whisper_errors(clean: str) -> str:
    """Simule les erreurs typiques produites par Whisper en FR."""
    raw = clean
    
    # Retirer la ponctuation finale (Whisper l'omet parfois)
    if random.random() < 0.4:
        raw = re.sub(r'[.!?]$', '', raw)
    
    # Minuscule initiale (Whisper parfois)
    if random.random() < 0.3 and raw:
        raw = raw[0].lower() + raw[1:]
    
    # Injecter des fillers au début
    if random.random() < 0.5:
        raw = random.choice(FILLERS) + raw
    
    # Injecter des fillers au milieu
    if random.random() < 0.3:
        words = raw.split()
        if len(words) > 3:
            pos = random.randint(1, len(words) - 1)
            filler = random.choice(FILLERS).strip()
            words.insert(pos, filler)
            raw = ' '.join(words)
    
    # Bégaiement (mot répété)
    if random.random() < 0.25:
        words = raw.split()
        if len(words) > 2:
            pos = random.randint(0, len(words) - 1)
            words.insert(pos + 1, words[pos])
            raw = ' '.join(words)
    
    # Espace après apostrophe (bug Whisper FR)
    if random.random() < 0.4:
        raw = re.sub(r"([jcnldJ])'([a-zA-ZÀ-ÿ])", r"\1' \2", raw)
    
    # Double ponctuation
    if random.random() < 0.15:
        raw = re.sub(r'([.?!])', r'\1\1', raw, count=1)
    
    return raw


def format_prompt(raw: str, mode: str = 'chat') -> str:
    """Format prompt instruction pour le fine-tuning."""
    mode_instructions = {
        'chat': 'Corrige ce texte transcrit par Whisper : supprime les fillers (euh, du coup, genre...), corrige les bégaiements, normalise la ponctuation et la majuscule initiale. Conserve le style et le ton naturel.',
        'pro':  'Reformule ce texte transcrit par Whisper en style professionnel : supprime les fillers, corrige les bégaiements, améliore la formulation pour un email ou document formel.',
        'code': 'Corrige ce texte transcrit par Whisper en préservant le vocabulaire technique. Supprime uniquement les fillers et bégaiements évidents.',
    }
    instruction = mode_instructions[mode]
    return f'<|im_start|>system\nTu es un assistant de correction de dictée vocale française.<|im_end|>\n<|im_start|>user\n{instruction}\n\nTexte brut : {raw}<|im_end|>\n<|im_start|>assistant\n'


# ── Génération ────────────────────────────────────────────────────────────────
random.seed(42)
N_SAMPLES = 10_000
dataset = []

modes = ['chat', 'chat', 'chat', 'pro', 'code']  # 60% chat, 20% pro, 20% code

for i in range(N_SAMPLES):
    clean = random.choice(CLEAN_SENTENCES)
    # Variations : combiner 2 phrases parfois
    if random.random() < 0.2:
        clean2 = random.choice(CLEAN_SENTENCES)
        if clean2 != clean:
            clean = clean + ' ' + clean2
    
    raw = inject_whisper_errors(clean)
    mode = random.choice(modes)
    
    prompt = format_prompt(raw, mode)
    full_text = prompt + clean + '<|im_end|>'
    
    dataset.append({
        'text': full_text,
        'raw': raw,
        'clean': clean,
        'mode': mode,
    })

# Mélanger et diviser train/val
random.shuffle(dataset)
split = int(len(dataset) * 0.95)
train_data = dataset[:split]
val_data   = dataset[split:]

print(f'✅ Dataset généré : {len(train_data)} train + {len(val_data)} val')
print(f'\nExemple :')
ex = train_data[0]
print(f'  Brut  : {ex["raw"]}')
print(f'  Propre: {ex["clean"]}')
print(f'  Mode  : {ex["mode"]}')

## 4. (Optionnel) Enrichir avec vos propres logs

Si vous avez des fichiers de logs réels de l'application, uploadez-les ici.
Format attendu : CSV avec colonnes `raw` et `clean`, ou JSON `[{"raw": ..., "clean": ...}]`.

In [ ]:
from google.colab import files
import json, csv

USE_REAL_LOGS = False  # Mettre True si vous avez des logs réels à uploader

if USE_REAL_LOGS:
    print('📁 Sélectionnez votre fichier de logs (.json ou .csv)')
    uploaded = files.upload()
    
    for filename, content in uploaded.items():
        if filename.endswith('.json'):
            real_data = json.loads(content)
        elif filename.endswith('.csv'):
            import io
            reader = csv.DictReader(io.StringIO(content.decode('utf-8')))
            real_data = list(reader)
        else:
            print(f'Format non supporté : {filename}')
            continue
        
        real_formatted = []
        for item in real_data:
            if 'raw' in item and 'clean' in item:
                mode = item.get('mode', 'chat')
                prompt = format_prompt(item['raw'], mode)
                real_formatted.append({'text': prompt + item['clean'] + '<|im_end|>', **item})
        
        # Les données réelles ont 3× plus de poids (répétées 3x)
        train_data.extend(real_formatted * 3)
        print(f'✅ {len(real_formatted)} exemples réels ajoutés (×3 = {len(real_formatted)*3} paires)')
    
    random.shuffle(train_data)
    print(f'Dataset total après enrichissement : {len(train_data)} exemples')
else:
    print('ℹ️  Entraînement sur dataset synthétique uniquement.')
    print('   Mettez USE_REAL_LOGS = True pour utiliser vos propres logs.')

## 5. Chargement du modèle de base (Task 21 — début)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Quantification 4-bit pour économiser la VRAM (T4 = 16 Go)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f'⏳ Chargement du tokenizer {MODEL_ID}...')
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

print(f'⏳ Chargement du modèle {MODEL_ID} en QLoRA 4-bit...')
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True,
)
model.config.use_cache = False

vram_used = torch.cuda.memory_allocated() / 1e9
print(f'✅ Modèle chargé — VRAM utilisée : {vram_used:.1f} Go')

## 6. Configuration LoRA et entraînement

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from datasets import Dataset

# Préparer le modèle pour le fine-tuning QLoRA
model = prepare_model_for_kbit_training(model)

# Configuration LoRA
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    lora_dropout=LORA_DROPOUT,
    bias='none',
    task_type='CAUSAL_LM',
)

model = get_peft_model(model, lora_config)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f'✅ LoRA configuré — paramètres entraînables : {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.1f}%)')

# Convertir en datasets HuggingFace
hf_train = Dataset.from_list([{'text': d['text']} for d in train_data])
hf_val   = Dataset.from_list([{'text': d['text']} for d in val_data])

# Configuration entraînement
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=50,
    eval_strategy='steps',
    eval_steps=200,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    warmup_ratio=0.03,
    lr_scheduler_type='cosine',
    report_to='none',
    max_seq_length=MAX_SEQ_LEN,
    dataset_text_field='text',
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_val,
    tokenizer=tokenizer,
)

print('⏳ Démarrage de l\'entraînement...')
print(f'   Epochs : {EPOCHS} | Batch effectif : {BATCH_SIZE * GRAD_ACCUM} | LR : {LEARNING_RATE}')
trainer.train()
print('✅ Entraînement terminé !')

## 7. Validation rapide du modèle

In [ ]:
from transformers import pipeline

model.eval()
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, 
                device_map='auto', max_new_tokens=128, temperature=0.0, do_sample=False)

test_cases = [
    ('euh du coup je voulais vous dire que genre c est vraiment important hein',   'chat'),
    ('j ai besoin d aide pour euh configurer mon ordinateur',                      'chat'),
    ('suite à notre échange je je vous adresse ce compte-rendu',                   'pro'),
    ('le le bug est lié à une mauvaise gestion des erreurs dans le module audio',  'code'),
]

print('── Validation ─────────────────────────────────────────────────────────')
for raw, mode in test_cases:
    prompt = format_prompt(raw, mode)
    output = pipe(prompt)[0]['generated_text']
    # Extraire uniquement la réponse de l'assistant
    response = output.split('<|im_start|>assistant\n')[-1].split('<|im_end|>')[0].strip()
    print(f'[{mode:4s}] Brut  : {raw}')
    print(f'       Corrigé : {response}')
    print()

## 8. Export GGUF Q4 (Task 22)

Fusionne les poids LoRA, exporte en GGUF Q4_K_M pour llama.cpp.

In [ ]:
from peft import PeftModel
import shutil

MERGED_DIR = '/content/qwen25-dictation-fr-merged'

# 1. Fusionner LoRA dans le modèle de base
print('⏳ Fusion des poids LoRA...')
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, torch_dtype=torch.float16, device_map='cpu', trust_remote_code=True
)
merged = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
merged = merged.merge_and_unload()
merged.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)
print(f'✅ Modèle fusionné sauvegardé dans {MERGED_DIR}')

# 2. Cloner llama.cpp pour la conversion
print('⏳ Installation llama.cpp pour conversion GGUF...')
!git clone -q --depth 1 https://github.com/ggerganov/llama.cpp /content/llama.cpp
!pip install -q -r /content/llama.cpp/requirements/requirements-convert_hf_to_gguf.txt

# 3. Conversion HuggingFace → GGUF F16
print('⏳ Conversion vers GGUF F16...')
!python /content/llama.cpp/convert_hf_to_gguf.py \
    {MERGED_DIR} \
    --outfile /content/qwen25-dictation-fr-f16.gguf \
    --outtype f16

# 4. Quantification Q4_K_M
print('⏳ Quantification Q4_K_M...')
!cd /content/llama.cpp && cmake -B build -DCMAKE_BUILD_TYPE=Release > /dev/null 2>&1
!cd /content/llama.cpp && cmake --build build --config Release -j $(nproc) > /dev/null 2>&1
!/content/llama.cpp/build/bin/llama-quantize \
    /content/qwen25-dictation-fr-f16.gguf \
    /content/{GGUF_NAME} \
    Q4_K_M

size_mb = os.path.getsize(f'/content/{GGUF_NAME}') / 1e6
print(f'\n✅ Export terminé : {GGUF_NAME} ({size_mb:.0f} Mo)')

## 9. Téléchargement du modèle fine-tuné

In [ ]:
from google.colab import files
import os

gguf_path = f'/content/{GGUF_NAME}'

if os.path.exists(gguf_path):
    size_mb = os.path.getsize(gguf_path) / 1e6
    print(f'📥 Téléchargement de {GGUF_NAME} ({size_mb:.0f} Mo)...')
    files.download(gguf_path)
    print()
    print('✅ Fichier téléchargé !')
    print()
    print('📋 Prochaine étape — intégration dans l\'app :')
    print(f'   mkdir -p ~/Library/Application\ Support/dictation-ia/models/')
    print(f'   mv ~/Downloads/{GGUF_NAME} ~/Library/Application\ Support/dictation-ia/models/')
    print()
    print('   Dans l\'app, vérifier que le modèle est détecté via Settings → LLM.')
else:
    print(f'❌ Fichier {gguf_path} introuvable. Vérifiez les erreurs dans la cellule précédente.')

---
## Résumé

| Étape | Description | Durée estimée |
|-------|-------------|---------------|
| Dataset | 10 000 paires synthétiques FR | ~30s |
| Chargement modèle | Qwen2.5-0.5B-Instruct en QLoRA 4-bit | ~3min |
| Entraînement | 3 epochs, batch effectif 32 | ~2-3h (T4) / ~45min (A100) |
| Export GGUF | Fusion LoRA + quantification Q4_K_M | ~15min |
| **Total** | | **~3-4h (T4)** |

**Taille du modèle final** : ~400 Mo (Q4_K_M)

**RAM requise dans l'app** : ~320 Mo (chargé à la demande)